In [37]:
%%file add.cpp
int add(int a, int b) {
    return a+b;
}

Overwriting add.cpp


In [38]:
%%file add.hpp
#ifndef ADD_HPP
#define ADD_HPP
int add(int a, int b);
#endif

Overwriting add.hpp


In [39]:
%%file driver.cpp
#include <iostream>
#include "add.hpp"
using std::cout;
using std::endl;

int main() {
    cout << add(3, 4) << endl;
}

Overwriting driver.cpp


In [40]:
%%bash
g++ -o driver.exe driver.cpp add.cpp
./driver.exe

7


In [41]:
%%file plus.pyx

cdef extern from 'add.cpp':
    int add(int a, int b)

def plus(a, b):
    return add(a, b)

Overwriting plus.pyx


In [45]:
%%file setup.py 
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize(
           "plus.pyx",   
           sources=["add.cpp"], 
           language="c++",        
      ))

Overwriting setup.py


In [46]:
%%bash
python setup.py build_ext --inplace

running build_ext


In [44]:
import plus

ImportError: dlopen(/Users/cliburn/git/sta-663-2016/lectures/plus.so, 2): Symbol not found: _PyString_Type
  Referenced from: /Users/cliburn/git/sta-663-2016/lectures/plus.so
  Expected in: dynamic lookup


In [30]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [28]:
%%cython


missing cimport in module 'add': /Users/cliburn/.ipython/cython/_cython_magic_bf893b21db7370db986f6cc004136f06.pyx



Error compiling Cython file:
------------------------------------------------------------
...
cimport add
       ^
------------------------------------------------------------

/Users/cliburn/.ipython/cython/_cython_magic_bf893b21db7370db986f6cc004136f06.pyx:1:8: 'add.pxd' not found
